In [1]:
import time
import matplotlib.pyplot as plt
import torch
import numpy as np
from laplacian_py import laplacian_solver

res = 30
max_val = 40
cost_scale = .02

values = .99*max_val*torch.ones((res, res, res),dtype=torch.float32, device='cuda')
cost = cost_scale*torch.ones((res,res, res),dtype=torch.float32, device='cuda')
boundary_conditions = torch.zeros((res,res, res),dtype=torch.float32, device='cuda')
boundary_types = torch.zeros((res,res, res),dtype=torch.float32, device='cuda')
# goal
goal_ind1 = int(res*15/30)
goal_ind2 = int(res*15/30)
goal_ind3 = int(res*15/30)
boundary_conditions[goal_ind1, goal_ind2, goal_ind3] = 0.0
boundary_types[goal_ind1, goal_ind2, goal_ind3] = 1.0



In [2]:
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim
from laplacian_py.network import LaplaceNet, compute_loss, calculate_gradient
    
network = LaplaceNet((res, res, res), max_val, 1*cost_scale, 0)
network.cuda(device='cuda')
V_in = values.clone()

print(network.indexes)
print(network.C.shape)
print(network.num_dims)

tensor([-931,  -31,  869, -901,   -1,  899, -871,   29,  929, -930,  -30,  870,
        -900,  900, -870,   30,  930, -929,  -29,  871, -899,    1,  901, -869,
          31,  931], device='cuda:0', dtype=torch.int32)
torch.Size([30, 30, 30])
3


In [4]:
lr = 1e-2 # norm
# lr = 1e-4 # abs
momentum = 0.99
optimizer = optim.SGD(network.parameters(), lr=lr, momentum=momentum)

XYZ_train = [np.linspace(-1, 1, int(res), dtype=np.float32) for _ in range(3)]
X_train, Y_train, Z_train = np.meshgrid(*XYZ_train)
out = np.stack([X_train, Y_train, Z_train], axis=3)
grid_train = torch.tensor(out, dtype=torch.float32, device='cuda', requires_grad=True)
grid_train = grid_train.unsqueeze(axis=0)


# target = torch.tensor(5*X_train+5*Y_train +5*Z_train + 15,dtype=torch.float32, device='cuda')
# target = torch.tensor(15*Z_train + 15,dtype=torch.float32, device='cuda')
target = torch.tensor(30*np.sqrt(X_train**2+Y_train**2),dtype=torch.float32, device='cuda')

# target = torch.tensor(20*X + 0,dtype=torch.float32, device='cuda')
# target = torch.tensor(12*(X**2+Y**2+Z**2) + 0,dtype=torch.float32, device='cuda')
# target = torch.tensor(15 + 0*X,dtype=torch.float32, device='cuda')
# target = torch.tensor(10*Y + 10*(1-X), dtype=torch.float32, device='cuda')
# target = torch.tensor(10*(1-Y) + 10*X, dtype=torch.float32, device='cuda')

# target = target.unsqueeze(axis=0)
network.reset()
for it in range(0, 2000):
    optimizer.zero_grad()
    pred, C, _ = network(grid_train, boundary_types, boundary_conditions)
    loss_train = 1*compute_loss(pred=pred, target=target, C=C, cost_scale=cost_scale)
    if it % 10 == 0:
        print(loss_train)
    loss_train.backward()
    optimizer.step()
    



tensor(242.8103, device='cuda:0', grad_fn=<MulBackward0>)
tensor(399.2968, device='cuda:0', grad_fn=<MulBackward0>)
tensor(329.2004, device='cuda:0', grad_fn=<MulBackward0>)
tensor(283.7846, device='cuda:0', grad_fn=<MulBackward0>)
tensor(270.7912, device='cuda:0', grad_fn=<MulBackward0>)
tensor(259.1895, device='cuda:0', grad_fn=<MulBackward0>)
tensor(247.6995, device='cuda:0', grad_fn=<MulBackward0>)
tensor(236.5297, device='cuda:0', grad_fn=<MulBackward0>)
tensor(225.7844, device='cuda:0', grad_fn=<MulBackward0>)
tensor(215.5126, device='cuda:0', grad_fn=<MulBackward0>)
tensor(205.7321, device='cuda:0', grad_fn=<MulBackward0>)
tensor(196.4430, device='cuda:0', grad_fn=<MulBackward0>)
tensor(187.6360, device='cuda:0', grad_fn=<MulBackward0>)
tensor(179.2956, device='cuda:0', grad_fn=<MulBackward0>)
tensor(171.4020, device='cuda:0', grad_fn=<MulBackward0>)
tensor(163.9338, device='cuda:0', grad_fn=<MulBackward0>)
tensor(156.8708, device='cuda:0', grad_fn=<MulBackward0>)
tensor(150.190

tensor(25.5150, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.5520, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.5920, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.6236, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.6642, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.7051, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.7466, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.7858, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.8298, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.8711, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.9140, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.9572, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25.9974, device='cuda:0', grad_fn=<MulBackward0>)
tensor(26.0435, device='cuda:0', grad_fn=<MulBackward0>)
tensor(26.0882, device='cuda:0', grad_fn=<MulBackward0>)
tensor(26.1266, device='cuda:0', grad_fn=<MulBackward0>)
tensor(26.1689, device='cuda:0', grad_fn=<MulBackward0>)
tensor(26.2134, device='cuda:0'

In [5]:
torch.max(network.V)
max_val

40

In [6]:
boundary_types_2 = boundary_types.clone()
boundary_conditions_2 = boundary_conditions.clone()


# obj1_ind1 = int(res*18/30)
# obj1_ind2 = int(res*2/3)
# obj1_ind3 = int(res*0)
# obj1_ind4 = int(res*2/3)
# boundary_conditions_2[obj1_ind1:obj1_ind2, obj1_ind3:obj1_ind4] = max_val*10
# boundary_types_2[obj1_ind1:obj1_ind2, obj1_ind3:obj1_ind4] = 1.0

# obj2_ind1 = int(res*3/12)
# obj2_ind2 = int(res*4/12)
# obj2_ind3 = int(res*1/3)
# obj2_ind4 = int(res*28/30)
# boundary_conditions_2[obj2_ind1:obj2_ind2, obj2_ind3:obj2_ind4] = max_val*10
# boundary_types_2[obj2_ind1:obj2_ind2, obj2_ind3:obj2_ind4] = 1.0


# obj1_ind1 = int(res*1/3)
# obj1_ind2 = int(res*2/3)
# obj1_ind3 = int(res*1/3)
# obj1_ind4 = int(res*2/3)
# boundary_conditions_2[obj1_ind1:obj1_ind2, obj1_ind3:obj1_ind4] = max_val*10
# boundary_types_2[obj1_ind1:obj1_ind2, obj1_ind3:obj1_ind4] = 1.0


# tmp_inds = boundary_types_2 > 0
# tmp_inds = tmp_inds*(torch.rand(tmp_inds.shape,dtype=torch.float32, device='cuda')>.03)
# boundary_types_2[tmp_inds] = 0
# boundary_conditions_2[tmp_inds] = 0
# boundary_conditions_2[goal_ind1, goal_ind2] = 0.0
# boundary_types_2[goal_ind1, goal_ind2] = 1.0

# network.reset()
for i in range(1000):
    pred, C_pos, J2 = network(grid_train, boundary_types_2, boundary_conditions_2)
    V_in = pred.detach()
 
pred[pred>max_val] = max_val





In [7]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
import matplotlib
matplotlib.use('TkAgg')

z = pred.cpu().detach().numpy()[1:-1, 1:-1, 1:-1]
X = np.linspace(-15, 15, z.shape[0])
Y = np.linspace(-15, 15, z.shape[1])
Z = np.linspace(-15, 15, z.shape[2])
X, Y, Z = np.meshgrid(X, Y, Z)


# # ax = plt.figure().add_subplot(projection='3d')
# ax = plt.figure(figsize=(20, 20)).add_subplot(projection='3d')

# Up = -np.diff(z[1:-2, 2:-2, 2:-2], axis=0)
# Vp = -np.diff(z[2:-2, 1:-2, 2:-2], axis=1)
# Wp = -np.diff(z[2:-2, 2:-2, 1:-2], axis=2)

# ax.quiver(X[2:-2, 2:-2, 2:-2], Y[2:-2,2:-2,2:-2], Z[2:-2,2:-2,2:-2], Vp, Up, Wp, length=0.5, normalize=True)

# ax.view_init(30, 30, 0)
# # f = plt.figure(figsize=(20, 20))
# plt.show()
# plt.pause(0)

In [8]:
# old way
dt = 0.01
ax = plt.figure().add_subplot(projection='3d')

grid_pred = pred[1:-1, 1:-1, 1:-1].unsqueeze(axis=0).unsqueeze(axis=0)

for x_val in [-.95, -.5, 0, .5 ,.95]:
    for y_val in [-.95, -.5, 0, .5 ,.95]:
        for z_val in [-.95, 0.0, .95]:
            cur = np.reshape(np.array([x_val, y_val, z_val]),(1, 1, 1, 1, 3))
            traj = np.zeros((1000, 3))
            for i in range(traj.shape[0]):
                query = torch.tensor(cur, dtype=torch.float32, device='cuda', requires_grad=True)
                query.unsqueeze(axis=0)
                J1, J2 = calculate_gradient(grid_pred, query)
                J2 = J2.cpu().detach().numpy()
                J2 = J2/(np.linalg.norm(J2) + .00000000001)
                cur += -J2*dt
#                 print(J2)
#                 print(cur)
                traj[i, :] = cur.squeeze()

            ax.plot(traj[:,0], traj[:,1], traj[:,2])
    
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()
plt.pause(0)


In [9]:
# dt = 0.01
# ax = plt.figure().add_subplot(projection='3d')

# for x_val in [-.99, -.5, 0, .5 ,.99]:
#     for y_val in [-.99, -.5, 0, .5 ,.99]:
#         for z_val in [.95]:
#             cur = np.reshape(np.array([x_val, y_val, z_val]),(1, 1, 1, 1, 3))
#             traj = np.zeros((1000, 3))
#             for i in range(traj.shape[0]):
#                 query = torch.tensor(cur, dtype=torch.float32, device='cuda', requires_grad=True)
#                 query.unsqueeze(axis=0)
#                 _, _, J2 = network(query, boundary_types_2, boundary_conditions_2)
#                 J2 = J2.cpu().detach().numpy()
#                 J2 = J2/(np.linalg.norm(J2) + .00000000001)
#                 cur += -J2*dt
#                 traj[i, :] = cur     

#             ax.plot(traj[:,0], traj[:,1], traj[:,2])

# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')
# plt.show()

# plt.pause(0)

In [10]:
network.V[2,2,2]
# boundary_types_2[2,2,2]

tensor(38., device='cuda:0')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
                               
val = pred[1:-1, 1:-1, 1:-1].cpu().detach().numpy()

skip = 1
color = val[0:-1:skip, 0:-1:skip, 0:-1:skip]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(X[0:-1:skip,0:-1:skip,0:-1:skip], Y[0:-1:skip,0:-1:skip,0:-1:skip], Z[0:-1:skip,0:-1:skip,0:-1:skip],
          c=color)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()
plt.pause(0)

In [12]:
X[0:-1:skip,0:-1:skip,0:-1:skip].shape
color.shape
X.shape
val.shape

(28, 28, 28)

In [13]:
import itertools
res_in = (res, res, res)
num_dims = len(res_in)
dims_linear = torch.zeros(num_dims, dtype=torch.int32)
base = 1
for ind, d in enumerate(res_in):
    dims_linear[ind] = base
    base = base * d

indexes = torch.tensor([torch.sum(torch.tensor(v, dtype=torch.int32) * dims_linear) for v in
                     list(itertools.product([-1, 1], repeat=num_dims)) if
                     not all(vi == 0 for vi in v)], dtype=torch.int32, device='cuda')

indexes

tensor([-931,  869, -871,  929, -929,  871, -869,  931], device='cuda:0',
       dtype=torch.int32)